# Loading Datasets

In [441]:
import pandas as pd
import json
import copy
import numpy as np

import plotly.graph_objs as go
from plotly.offline import iplot

In [442]:
portfolio_df = pd.read_csv('../Datasets/portfolio.csv', index_col=0)
profile_df = pd.read_csv('../Datasets/profile.csv', index_col=0)
transcript_df = pd.read_csv('../Datasets/transcript.csv', index_col=0)

In [443]:
portfolio_df.info()
portfolio_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [444]:
# columns gender and income has null values, we will see how to handle them later
profile_df.info()
profile_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 796.9+ KB


,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [445]:
transcript_df.info()
transcript_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 11.7+ MB


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


# Merging datasets

In [446]:
# checking for duplicated profile information
print("Are there any duplicated profiles in profile dataframe?", profile_df.duplicated('id').any())

Are there any duplicated profiles in profile dataframe? False


In [447]:
transcript_profile_df = pd.merge(transcript_df, profile_df, left_on='person', right_on='id', how='outer')

In [448]:
value_count_for_value = {}

for item in transcript_profile_df['value']:
    data = json.loads(item.replace("'", '"'))
    for key in data.keys():
        if key in value_count_for_value:
            value_count_for_value[key] += 1
        else:
            value_count_for_value[key] = 1

print("Types of values in value column of dataframe", value_count_for_value)

"""
Looking at a csv dataset record, "12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,"{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2}",0"
difference between 'offer_id' and 'offer id' is that when there is a offer completed, it will be 'offer_id' and a reward, instead of 'offer id'.
Let us validate that below
"""

mapping_flag = True
for index, record in transcript_profile_df.iterrows():
    value = record['value']
    event = record['event']
    if 'offer_id' in value:
        if event == 'offer completed':
            pass
        else:
            print("Invalidated. 'offer_id' is not a 1-1 mapping with offer completed event.", record)
            mapping_flag = False
            break

if mapping_flag:
    print("Validated. 'offer_id' is a 1-1 mapping with offer completed event.")

Types of values in value column of dataframe {'offer id': 134002, 'amount': 138953, 'offer_id': 33579, 'reward': 33579}


Validated. 'offer_id' is a 1-1 mapping with offer completed event.


In [449]:
def extract_values(x):
    data = json.loads(x.replace("'", '"'))
    
    offer_id = data.get('offer id')
    transaction_amount = data.get('amount')
    reward_awarded = data.get('reward')

    if reward_awarded:
        offer_id = data.get('offer_id')

    return offer_id, transaction_amount, reward_awarded

transcript_profile_df[['offer_id', 'transaction_amount', 'reward_awarded']] = transcript_profile_df['value'].apply(lambda x: extract_values(x)).apply(pd.Series)


In [450]:
full_df = pd.merge(transcript_profile_df, portfolio_df, left_on='offer_id', right_on='id', how='outer')

# Data Cleaning and Transformation

In [451]:
#dropping columns because id_x is person id and id_y is offer_id
pre_clean_df = copy.deepcopy(full_df)
pre_clean_df = pre_clean_df.drop(['id_x', 'id_y', 'value'], axis=1)

# converting became_member_on to date time format
pre_clean_df['became_member_on'] = pd.to_datetime(pre_clean_df['became_member_on'], format='%Y%m%d')
profile_df['became_member_on'] = pd.to_datetime(profile_df['became_member_on'], format='%Y%m%d')

In [452]:
#handling null in gender and income columns

#is gender null when income is null?
gender_null_df = pre_clean_df[pre_clean_df['gender'].isnull()]
both_null = gender_null_df['income'].isnull().all()

print("All income is null when gender is null?", both_null)
print("How many null rows?", len(gender_null_df))
print("How many percentage of null gender and income contributes to all transactions?", round(len(gender_null_df)/len(pre_clean_df)*100, 2))
print("How many customers did not indicate gender?", round(len(profile_df[profile_df["gender"].isnull()])/len(profile_df)*100, 2))

All income is null when gender is null? True
How many null rows? 33772
How many percentage of null gender and income contributes to all transactions? 11.02
How many customers did not indicate gender? 12.79


Since income and gender are crucial in developing an accurate profile of a person, we could either:
1. Remove every row with income and gender is null since 11% of transactions removed should not cause much lost in information
2. Remove gender and income columns entirely and replace them with binary variables that indicates if the user provided these information (eg. 0 if not provided)
3. Create 2 datasets, one with and one without gender and income and conduct separate analysis on each of them

There is no additional information that would allow us to accurately determine the income of those who did not indicate it, hence it would be best to remove
the income column. As for gender, we can create a new variable "Unknown" for those who do not indicate it. However, since gender is null when income is null,
we will remove all rows where gender and income is null.

In [453]:
pre_clean_df = pre_clean_df[pre_clean_df['income'].notna()]
cleaned_profile_df = profile_df[profile_df['income'].notna()]

# changing string list to a list
portfolio_df['channels'] = portfolio_df['channels'].apply(lambda x: eval(x))
cleaned_portfolio_df = copy.deepcopy(portfolio_df)

In [454]:
#new cleaned df

cleaned_df = copy.deepcopy(pre_clean_df)

# Exploratory Data Analysis

In [455]:
cleaned_profile_df.head()

,gender,age,id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0


In [456]:
cleaned_df.head()

,person,event,time,gender,age,became_member_on,income,offer_id,transaction_amount,reward_awarded,reward,channels,difficulty,duration,offer_type
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,6,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,132,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,5.0,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo
3,e2127556f4f64592b11af22de27a7932,offer received,408,M,68,2018-04-26,70000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo
4,e2127556f4f64592b11af22de27a7932,offer viewed,420,M,68,2018-04-26,70000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo


In [457]:
# template for graphs

graph_custom_template = dict(
    layout=go.Layout(
        title_font=dict(size=24),
        xaxis=dict(
            title_font=dict(size=18),
            showgrid=True,
            gridwidth=1,
            showline=True,
            linecolor='black',
            linewidth=2,
            gridcolor='Grey'
        ),
        yaxis=dict(
            title_font=dict(size=18),
            showgrid=True,
            gridwidth=1,
            showline=True,
            linecolor='black',
            linewidth=2,
            gridcolor='Grey'
        )
    )
)

## Univariate & Bivariate Analysis

### Gender Analysis

In [458]:
# gender distribution plot
gender_counts = cleaned_profile_df['gender'].value_counts()
gender_data = go.Pie(labels=gender_counts.index, values=gender_counts.values)

fig = go.Figure(data=[gender_data])
fig.update_layout(title_text='Gender Distribution', template=graph_custom_template)
fig.show()

**Insights:**

Purchase amount during the period are mainly by male and females with males contributing to 20%~ more purchases than females. A small percentaage of 1.5%~ have others as their gender.

### Age Analysis

In [459]:
# age distribution plot
fig = go.Figure(data=[go.Histogram(
    x=cleaned_profile_df['age'],
    marker=dict(
        color='rgba(0, 123, 255, 0.7)',
        line=dict(
            color='rgba(0, 123, 255, 1)',
            width=2  
        )
    )
)])

fig.update_layout(
    title_text='Age Distribution',
    xaxis_title_text='Age', 
    yaxis_title_text='Count',
    template=graph_custom_template,
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()

bin_size = 10
bins = np.arange(10, cleaned_profile_df['age'].max() + bin_size, bin_size)
hist, bin_edges = np.histogram(cleaned_profile_df['age'], bins=bins)

percentages = (hist / hist.sum()) * 100
bin_labels = [f"{edge} - {edge+bin_size}" for edge in bins[:-1]]


fig = go.Figure(data=go.Bar(
    x=bin_labels,
    y=percentages,
    text=np.round(percentages, 2),
    textposition='outside',
    marker=dict(
        color='rgba(0, 123, 255, 0.7)',
        line=dict(
            color='rgba(0, 123, 255, 1)',
            width=2  
        )
    )
))

fig.update_layout(
    title_text='Age Distribution with Percentage Contribution',
    xaxis_title_text='Age Bins',
    yaxis_title_text='Percentage of Total',
    template=graph_custom_template,
)

fig.show()


**Insights:**

Customers aged 10 to 70 make up 80% Starbucks customer base.
Customers aged 50 to 60 make up 23.89% of Starbucks customer base which is the most, compared to other age groups. This is followed by age group 60 to 70 which makes up 20.18% and 40 to 50, which makes up 15.58%.

### Income Analysis

In [460]:
fig = go.Figure(data=[go.Histogram(
    x=cleaned_profile_df['income'],
    marker=dict(
        color='rgba(0, 123, 255, 0.7)',
        line=dict(
            color='rgba(0, 123, 255, 1)',
            width=2  
        )
    )
)])

fig.update_layout(
    title_text='Income Distribution',
    xaxis_title_text='Income', 
    yaxis_title_text='Count',  
    template=graph_custom_template,
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()

**Insights:**

The most common income range among the members is between $50k and $70k as seen from the tallest bars being within that income range. As income increases beyond the $70k mark, the number of membes within each income range decreases.

Lowest income members: On the left of the graph, there's a sudden jump in member Count starting from around $30k, which suggests that there are relatively fewer members with income lower than $30k.

### Age VS Income Analysis

In [461]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Histogram2dContour(
    x=cleaned_profile_df['age'], 
    y=cleaned_profile_df['income'],
    colorscale='sunsetdark',
    contours=dict(
        showlabels=True,
        labelfont=dict(
            size=12,
            color='black',
        )
    )
))

fig.update_layout(
    title_text='Income-Age Density Distribution',
    xaxis_title_text='Age', 
    yaxis_title_text='Income',
    template=graph_custom_template
)

fig.show()


**Insights on the relationship between Income and Age:**

The highest data concentration seems to be within the age range of roughly 45 to 65. This is indicated by the densest contour lines labelled with higher numbers like 120 to 160, which that the region enclosed by the line contains 120 to 160 data points respectively.

The common income range for this densest age group is around 60k to 80k as this is where the contours are closest together. As the age increases beyond 65 or decreases below 45, the density of data points decreases, as indicated by the wider spacing between contour lines.

Younger Age Group: There's a lower density of data points those under 45, which is indicated by the more sparse contour lines in the left area.

High-Income Earners: There are individuals in the higher income brackets (above 100k), especially in the age range of 55 to 65.

There appears to be little relationship between age and income as there no specific trends to the contours. However, the highest income levels do seem to tbe clustered around the middle aged group.

### Age VS Gender Analysis

In [462]:
fig1 = go.Figure()
fig1.add_trace(go.Box(x=cleaned_profile_df['gender'], y=cleaned_profile_df['age'], boxmean='sd'))
fig1.update_layout(
    title='Age Distribution by Gender',
    xaxis_title='Gender',
    yaxis_title='Age',
    template= graph_custom_template
)
fig1.show()

avg_age_by_gender = cleaned_profile_df.groupby('gender')['age'].mean().reset_index()
print("Average Age by Gender")
print(avg_age_by_gender)

Average Age by Gender
  gender        age
0      F  57.544950
1      M  52.116690
2      O  54.400943


**Insights:**

The average ages of female, male and other members are 57, 52 and 54 respectively. The spread of male members are wider than others and females which indicate a wider variety of male ages. All 3 genders follow a somewhat normal distribution as seen by their first and third quartile and median centered symmetrically around the mean.

There is also a presence of a outlier in the female gender of age 101. However, it does not skew the distribution by much.

### Membership Duration Analysis

In [463]:
cleaned_profile_df['became_member_on'] = pd.to_datetime(cleaned_profile_df['became_member_on'])
cleaned_profile_df['membership_year'] = cleaned_profile_df['became_member_on'].dt.year

# Membership trend over time
memberships_by_year = cleaned_profile_df.groupby('membership_year').size()
fig1 = go.Figure(data=[
    go.Bar(x=memberships_by_year.index, y=memberships_by_year.values)
])
fig1.update_layout(title='New Memberships per Year',
                   xaxis_title='Year',
                   yaxis_title='Count of New Memberships',
                   template= graph_custom_template)
fig1.show()

# Box plot for income by membership year
fig2 = go.Figure()
for year in sorted(cleaned_profile_df['membership_year'].unique()):
    fig2.add_trace(go.Box(
        y=cleaned_profile_df[cleaned_profile_df['membership_year'] == year]['income'],
        name=str(year),
        boxmean='sd'
    ))
fig2.update_layout(title='Income Distribution by Membership Year',
                   xaxis_title='Year',
                   yaxis_title='Income',
                   template=graph_custom_template)
fig2.show()

# Box plot for age by membership year
fig3 = go.Figure()
for year in sorted(cleaned_profile_df['membership_year'].unique()):
    fig3.add_trace(go.Box(
        y=cleaned_profile_df[cleaned_profile_df['membership_year'] == year]['age'],
        name=str(year),
        boxmean='sd'
    ))
fig3.update_layout(title='Age Distribution by Membership Year',
                   xaxis_title='Year',
                   yaxis_title='Age',
                   template=graph_custom_template)
fig3.show()

**Insights:**

The customers in this pool mainly signed up for a membership during 2016 to 2018. Before 2016, memebership sign ups were very low. This could mean that membership sign ups were somehow encouraged or pushed for after 2015.

Even though the number of sign ups increase by 3.5 times as compared from 2017 to 20115, the income and age distribution of the customers does not seem to vary much throughout the years. This means that the number of sign ups does not affect the demographics of Starbucks members as it seems that Starbucks has been true to its image in attracting a certain demographic of customers to sign up for their memberships.

### Offer Analysis

In [464]:
print("Distribution of Offer Types")
offer_type_counts = portfolio_df['offer_type'].value_counts()
print(offer_type_counts)

# Scatter Plot for Difficulty vs. Reward by Offer Type
fig2 = go.Figure()
for offer_type in portfolio_df['offer_type'].unique():
    df_subset = portfolio_df[portfolio_df['offer_type'] == offer_type]
    fig2.add_trace(go.Scatter(
        x=df_subset['difficulty'],
        y=df_subset['reward'],
        mode='markers',
        name=offer_type,
        marker=dict(
            size=12
        )
    ))

fig2.update_layout(
    title='Difficulty vs. Reward by Offer Type',
    xaxis_title='Difficulty',
    yaxis_title='Reward',
    legend_title='Offer Type',
    template=graph_custom_template
)
fig2.show()

Distribution of Offer Types
bogo             4
discount         4
informational    2
Name: offer_type, dtype: int64


**Insights:**

For informational offers, it is treated as 0 difficulty and 0 reward by Starbucks.
For buy one get one offers, the difficulty of the ofer is proportional to the rewards given.
For discount offers, the difficulty of the offer does not determine how much rewards the member will get when completing the offer.

In [468]:
channel_counts_per_offer_type = pd.DataFrame(index=cleaned_portfolio_df['offer_type'].unique())

for index, row in cleaned_portfolio_df.iterrows():
    offer_type = row['offer_type']
    for channel in row['channels']:
        if channel in channel_counts_per_offer_type.columns:
            channel_counts_per_offer_type.loc[offer_type, channel] = (
                channel_counts_per_offer_type.loc[offer_type, channel] + 1
            ) if pd.notna(channel_counts_per_offer_type.loc[offer_type, channel]) else 1
        else:
            channel_counts_per_offer_type[channel] = pd.NA
            channel_counts_per_offer_type.loc[offer_type, channel] = 1

channel_counts_per_offer_type.fillna(0, inplace=True)
channel_counts_per_offer_type = channel_counts_per_offer_type.astype(int)
channel_counts_per_offer_type.reset_index(inplace=True)
channel_counts_per_offer_type.rename(columns={'index': 'offer_type'}, inplace=True)

print("Total count of channels used by each offer:")
print(channel_counts_per_offer_type)

Total count of channels used by each offer:
      offer_type  email  mobile  social  web
0           bogo      4       4       3    3
1  informational      2       2       1    1
2       discount      4       3       2    4


**Insights:**

Distribution of Offer Types:
1. bogo:             4
2. discount:         4
3. informational:    2

Offer type bogo and discount mainly utilize email, mobile and web channels to reach out to customers. Whereas informational offer type mainly use email and mobile to reach out to customers.

In [467]:
fig3 = go.Figure()
for offer_type in portfolio_df['offer_type'].unique():
    df_subset = portfolio_df[portfolio_df['offer_type'] == offer_type]
    fig3.add_trace(go.Scatter3d(
        x=df_subset['difficulty'],
        y=df_subset['reward'],
        z=df_subset['duration'],
        mode='markers',
        name=offer_type,
        marker=dict(
            size=5, 
            opacity=0.8
        )
    ))

fig3.update_layout(
    title='Relationship between Duration, Reward, and Difficulty by Offer Type',
    scene=dict(
        xaxis_title='Difficulty',
        yaxis_title='Reward',
        zaxis_title='Duration (days)'
    ),
    legend_title='Offer Type',
    template=graph_custom_template,
    height= 600
)

fig3.show()


![alt text](image.png)
![alt text](image-1.png)

**Insights:**

Offers with higher difficulties like discount, gets a longer duration for completion.

Given the same difficulty and duration, bogo gives the best reward to difficult ratio and longest duration to difficulty ratio.

## Transactions Analysis

In [477]:
event_counts = cleaned_df['event'].value_counts()

fig = go.Figure(data=[go.Bar(
    x=event_counts.index,
    y=event_counts.values,
    text=event_counts.values,
    textposition='auto'
)])

fig.update_layout(
    title='Event Value Counts',
    xaxis_title='Event',
    yaxis_title='Count',
    template=graph_custom_template
)

fig.show()


In [479]:
# is transaction mappable to the a offer completed?

cleaned_df.head()

,person,event,time,gender,age,became_member_on,income,offer_id,transaction_amount,reward_awarded,reward,channels,difficulty,duration,offer_type
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,6,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,132,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,5.0,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo
3,e2127556f4f64592b11af22de27a7932,offer received,408,M,68,2018-04-26,70000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo
4,e2127556f4f64592b11af22de27a7932,offer viewed,420,M,68,2018-04-26,70000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,"['web', 'email', 'mobile']",5.0,7.0,bogo


**Insights:**

Transactions make up almost 45% of the all the dataset. Transactions is 4 times the number of offers completed which means that 